In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import os

import sys
sys.path.append("..")
from utils.serialization import save_model, load_model

### Dataset

In [2]:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

### Grid Search on Random Forest

In [3]:
pipe_rf = Pipeline([('pca', PCA()), ('rf', RandomForestClassifier())])
param_grid = [
    {
        'pca__n_components': range(1, X_train.shape[1] + 1, 2),
        'rf__n_estimators': range(10, 100, 5),
        'rf__criterion': ['gini', 'entropy'],
        'rf__max_depth': range(5, 10),
    }
]
gs = GridSearchCV(
    estimator=pipe_rf,
    param_grid=param_grid,
    scoring='accuracy'
)

gs.fit(X_train, y_train)
# print(pd.DataFrame(gs.cv_results_).head(10))
print(f"best_params: {gs.best_params_}\nbest_acc: {gs.best_score_:.4f} %")

best_params: {'pca__n_components': 3, 'rf__criterion': 'gini', 'rf__max_depth': 9, 'rf__n_estimators': 30}
best_acc: 0.9481 %


### Evaluation

In [4]:
prediction = gs.predict(X_test)
acc = accuracy_score(y_test, prediction)
recall = recall_score(y_test, prediction, average='macro')
precision = precision_score(y_test, prediction, average='macro')
f1 = f1_score(y_test, prediction, average='macro')
print(f"accuracy: {acc * 100:.1f} %")
print(f"recall: {recall * 100:.1f} %")
print(f"precision: {precision * 100:.1f} %")
print(f"f1_score: {f1 * 100:.1f} %")

accuracy: 86.7 %
recall: 86.7 %
precision: 90.5 %
f1_score: 86.1 %


### Save Model

In [5]:
path = os.path.join(os.getcwd(), "trained_models/model_Random_Forest.pickle")
save_model(path, gs)

### Load Model

In [6]:
path = os.path.join(os.getcwd(), "trained_models/model_Random_Forest.pickle")
model = load_model(path)
model.predict(X_test)

array([2, 0, 2, 2, 2, 2, 0, 0, 1, 2, 0, 1, 0, 2, 1])